In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from google.colab import files
from google.colab import drive
import glob
import zipfile
import os
import copy
from collections import Counter, defaultdict

In [ ]:
# Open a file upload dialog
# Select here all files to upload!
# If already uploaded, just press 'Cancel Upload'
# Note that we are here uploading all the english comments that we have past filtering phase.
# They are contained within a zip file.
uploaded = files.upload()

Saving english_data.zip to english_data.zip


In [ ]:
# Set the path to the data
# On local machine use the relative path, for example
# path = 'NLP labelled data preview/english set/'
# On Google Colab use this path
# '/content/'
path = '/content/'

In [ ]:
# Unzip the folder
with zipfile.ZipFile(path + 'english_data.zip', 'r') as zip_ref:
    zip_ref.extractall('english_data')

In [ ]:
# Load the dataset
# For Mac users : do english_data/english_data/*.csv
# For Windows users : do english_data/*.csv
all_english_comments = glob.glob(path + 'english_data/english_data/*.csv')

In [ ]:
# Read in the data
all_english_comments = pd.concat([pd.read_csv(f) for f in all_english_comments], ignore_index = True)

In [ ]:
# TODO : CHANGE LOCATION OF THESE STEPS !!

# Remove comments with words like "video" and "channel" as they are associated with comments such as 'great video!'
all_english_comments = all_english_comments[~all_english_comments['Comment'].str.contains('video|channel', case=False)]

# Remove comments whose length is less than 3 words
all_english_comments = all_english_comments[all_english_comments['Comment'].str.split().str.len() > 3]


In [ ]:
# Now we prepare for the labelling phase using a pre-trained state-of-the-art model

# Turn dataframe into a list
comments = all_english_comments['Comment'].tolist()

# Turn all comments into strings
comments = [str(comment) for comment in comments]

In [ ]:
# Load the different models, trained on different datasets
tokenizer_1 = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model_1 = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

# TODO : change to correct model, just for testing purposes
tokenizer_2 = AutoTokenizer.from_pretrained("aychang/roberta-base-imdb")
model_2 = AutoModelForSequenceClassification.from_pretrained("aychang/roberta-base-imdb")

tokenizer_3 = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
model_3 = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
# Move the models to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1.to(device)
model_2.to(device)
model_3.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [ ]:
# Initialize the pipelines
classifier_1 = pipeline('sentiment-analysis', model=model_1, tokenizer=tokenizer_1)
classifier_2 = pipeline('sentiment-analysis', model=model_2, tokenizer=tokenizer_2)
classifier_3 = pipeline('sentiment-analysis', model=model_3, tokenizer=tokenizer_3)


In [ ]:
# Predict sentiment labels for each classifier
predictions_1 = classifier_1(comments)
predictions_2 = classifier_2(comments)
predictions_3 = classifier_3(comments)

In [ ]:
# Extract the scores from the predictions
scores_1 = [prediction['score'] for prediction in predictions_1]
scores_2 = [prediction['score'] for prediction in predictions_2]
scores_3 = [prediction['score'] for prediction in predictions_3]
# Extract the labels from the predictions
labels_1 = [prediction['label'] for prediction in predictions_1]
labels_2 = [prediction['label'] for prediction in predictions_2]
labels_3 = [prediction['label'] for prediction in predictions_3]



In [ ]:
# Set up the right labels for the different models
# We want to transform all labels to the same format ; all should be numbers where 0 is negative, 1 is neutral and 2 is positive
# Model 1 gives Negative, Neutral, Positive as labels, so we will transform them to 0, 1, 2
labels_1 = [0 if label == 'Negative' else 1 if label == 'Neutral' else 2 for label in labels_1]
# Model 2 gives neg and pos as labels, so we will transform them to 0, 2
labels_2 = [0 if label == 'neg' else 2 for label in labels_2]
# Model 3 gives only POSITIVE, NEGATIVE as labels, so we will transform them to 0, 1
labels_3 = [0 if label == 'NEGATIVE' else 2 for label in labels_3]




In [ ]:
# Only keep comments with a confidence score of above 0.80
conf_score = 0.80
high_confidence_predictions_1 = []
high_confidence_comments_1 = []
high_confidence_comments_2 = []
high_confidence_predictions_2 = []
high_confidence_predictions_3 = []
high_confidence_comments_3 = []


In [ ]:
# For model 1 :
for i in range(len(scores_1)):
    if scores_1[i] > conf_score:
        high_confidence_predictions_1.append(labels_1[i])
        high_confidence_comments_1.append(comments[i])



In [ ]:
# For model 2 :
for i in range(len(scores_2)):
    if scores_2[i] > conf_score:
        high_confidence_predictions_2.append(labels_2[i])
        high_confidence_comments_2.append(comments[i])

In [ ]:
# For model 3 :
for i in range(len(scores_3)):
    if scores_3[i] > conf_score:
        high_confidence_predictions_3.append(labels_3[i])
        high_confidence_comments_3.append(comments[i])

In [ ]:
# MULTIPLE LABEL IDEA : This code still keeps only the comments that appear in atleast 2 models, but here
# it keeps all the labels, i.e. if the same label appeared only in 2 models and another label in the third,
# the label will look like [0,2]. We could just relabel these to introduce our bias

# Combining all comments and their respective labels
combined_comments = high_confidence_comments_1 + high_confidence_comments_2 + high_confidence_comments_3
combined_predictions = high_confidence_predictions_1 + high_confidence_predictions_2 + high_confidence_predictions_3

# Counting the occurrence of each comment
comment_counter = Counter(combined_comments)

# Filtering comments that appear in at least two lists
filtered_comments = [comment for comment, count in comment_counter.items() if count >= 2]

# Removing duplicates while keeping labels
unique_comments_with_labels = {}
for comment, label in zip(combined_comments, combined_predictions):
    if comment in filtered_comments:
        if comment not in unique_comments_with_labels:
            unique_comments_with_labels[comment] = []
        if label not in unique_comments_with_labels[comment]:
            unique_comments_with_labels[comment].append(label)

# Final unique comments with their labels
final_comments = list(unique_comments_with_labels.keys())
final_labels = list(unique_comments_with_labels.values())

print("Filtered Comments:", final_comments)
print("Respective Labels:", final_labels)

# Create a list out of result_labels, which only contains the elements that have length 1

sure_labels = []
unsure_labels = []
for labels in final_labels:
    if len(labels) == 1:
        sure_labels.append(labels[0])
    else:
        unsure_labels.append(labels[0])

# Check how many predictions we have in the respective classes
print("We have ", sure_labels.count(0), " negative predictions.")
print("We have ", sure_labels.count(1), " neutral predictions.")
print("We have ", sure_labels.count(2), " positive predictions.")

# Check how many unsure predictions we have
print("We have ", len(unsure_labels), " unsure predictions.")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# ONE LABEL IDEA
"""
# Combining all comments and their respective labels
combined_comments = high_confidence_comments_1 + high_confidence_comments_2 + high_confidence_comments_3
combined_predictions = high_confidence_predictions_1 + high_confidence_predictions_2 + high_confidence_predictions_3

# Counting the occurrence of each comment
comment_counter = Counter(combined_comments)

# Filtering comments that appear in at least two lists
filtered_comments = {comment for comment, count in comment_counter.items() if count >= 2}

# Dictionary to keep track of labels for each comment
comment_labels = defaultdict(list)

# Populate the dictionary with labels for each comment
for comment, label in zip(combined_comments, combined_predictions):
    if comment in filtered_comments:
        comment_labels[comment].append(label)

# Keep only labels that appear at least twice for each comment
final_comments = []
final_labels = []
for comment, labels in comment_labels.items():
    label_count = Counter(labels)
    filtered_labels = [label for label, count in label_count.items() if count >= 2]
    if filtered_labels:
        final_comments.append(comment)
        final_labels.append(filtered_labels)

print("Filtered Comments:", final_comments)
print("Respective Labels:", final_labels)


# Check how many predictions we have in the respective classes
print("We have ", final_labels.count(0), " negative predictions.")
print("We have ", final_labels.count(1), " neutral predictions.")
print("We have ", final_labels.count(2), " positive predictions.")
"""

In [ ]:
# Check how many comments are left after filtering by confidence score
print("We have ", len(final_comments), " comments left after filtering by confidence score " , conf_score , " .")


In [ ]:
# Save to csv the comments and their label
high_confidence_comments_df = pd.DataFrame(final_comments, columns=['comments'])
high_confidence_comments_df['predictions'] = final_labels
high_confidence_comments_df.to_csv(path + "High_Confidence_Comments_English.csv")

In [ ]:
# Download the file to your local machine (from google colab)

files.download(path + "High_Confidence_Comments_English.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>